In [1]:
# Import libraries, define data zones and time ranges, and authenticate
import polars as pl
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from api_wrapper.user_params import *
from api_wrapper.altitude_params import *
from api_wrapper import analysis_functions
import api_wrapper.altitude_functions as altitude

In [ ]:
#getRegionalDomicileAnalytics example with standard zones (list of counties for county-by-county results). Results will be separated by zone.
params = defaultRegionalDomicileParams
params['zones'] = [richmond_zones]
params['dateRange'] = months[0]
params['vehicleClasses'] = classes_and_schemes[0]['vehicleClasses']
params['vehicleClassSchemeId'] = classes_and_schemes[0]['vehicleClassSchemeId']
params['vocations'] = [1]
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, out_path = out_path, serialize = True, process = True)
result_df = job_outputs[0].results

In [ ]:
#getRegionalDomicileAnalytics example with custom zone (Richmond analysis region counties). Results will be aggregated across the whole region.
params = defaultRegionalDomicileParams
params['zones'] = [richmond_grouped_zone_id]
params['dateRange'] = months[0]
params['vehicleClasses'] = classes_and_schemes[0]['vehicleClasses']
params['vehicleClassSchemeId'] = classes_and_schemes[0]['vehicleClassSchemeId']
params['vocations'] = [1]
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, out_path = out_path, serialize = True, process = True)
results = job_outputs[0].results
subzone_info = job_outputs[0].subzone_info
query_info = job_outputs[0].query_info
# OD Results Analysis
stop_result_percentiles = (query_info
    .join(results, on="Query_ID", how="inner")
    .join(subzone_info, how="inner", on=["Query_ID", "Subzone"])
    .filter(pl.col("Group_By")=="None",
            pl.col("Statistic") == "Percentile")
    .pipe(analysis_functions.getWeibullFit, measure_names=["DomDCDistance"])
).collect()